In [1]:
#Importing necessary libraries
import numpy as np                  #For arryas
import json                         #For JSON data
import os                           #For OS dependent functionalities like making directories
from tqdm import tqdm, trange       #For progress bars
import h5py                         #For working with hash files
import csv                          #To read and write tabular data in CSV format
import cv2                          #For importing opencv-python, a great tool for image processing and performing computer vision tasks
import matplotlib.pyplot as plt     #For data visualisation
import seaborn as sns               #For data visualisation
import mat73                        #To load MATLAB 7.3 HDF5 files into a Python dictionary
import tensorflow as tf             #For training machine learning models 
import tensorflow.keras as Keras    #For training machine learning models 
from tensorflow.keras.models import Sequential                                                  #A Sequential model is appropriate for a plain stack of layers where each layer has exactly one input tensor and one output tensor
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, TimeDistributed, Concatenate    #Other layers of the model
from tensorflow.keras.layers import Attention                                                   #Other layers of the model
from tensorflow.keras import Input, Model                                                       #Other layers of the model                                                
from sklearn.model_selection import train_test_split                                            #Splitting training and testing dataset
from prettytable import PrettyTable                                                             #Formatting the table
import evals                                                                                    #For Evaluation metrics

In [2]:
'''
Data generators:
Even the most state-of-the-art configuration may not have enough memory space to process extremely 
huge datasets the way we used to do it. That is why we need data generators to generate the dataset 
on multiple cores in real time and feed it right away to the deep learning model efficiently.
'''
class DataGenerator(Keras.utils.Sequence):

    '''
    This is the initialization function of the class. We make the latter inherit the properties 
    of keras.utils.Sequence so that we can leverage several functionalities.

    We put as arguments relevant information about the data, such as dimension sizes, 
    batch size, or decide whether we want to shuffle our data at generation.
    Here, the batch size is set to 5 and shuffle is set to false, which means
    we will not get a new order of exploration at each pass.
    '''
    def __init__(self,dataset,batch_size=5,shuffle=False):

        self.dataset = dataset
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()                 #The method on_epoch_end is triggered once at the very beginning and once at the end of each epoch


    '''
    This function denotes the number of batches per epoch.
    A common practice is to set this value to [# of samples / batch-size ]
    so that the model sees the training samples at most once per epoch.
    '''
    def __len__(self):

        return int(np.floor(len(self.dataset)/self.batch_size))

    '''
    Now, when the batch corresponding to a given index is called, the generator 
    executes the __getitem__ method to generate one batch of data.
    '''
    def __getitem__(self,index):

        indexes = self.indices[index * self.batch_size : (index+1) * self.batch_size]   # Generate indexes of the batch
        feature, label = self.__data_generation(indexes)                                # Generate data

        return feature, label

    '''
    Another method that is core to the generation process is the one that 
    achieves the most crucial job: producing batches of data. The private method 
    in charge of this task is called __data_generation and takes as argument the 
    list of IDs of the target batch.
    '''
    def __data_generation(self,indexes):

        # Initialization
        feature = np.empty((self.batch_size,320,1024))
        label = np.empty((self.batch_size,320,1))

        # Generate data
        for i in range(len(indexes)):
            feature[i,] = np.array(self.dataset[indexes[i]][0])                 # Store sample
            label[i,] = np.array(self.dataset[indexes[i]][1]).reshape(-1,1)     # Store class

        return feature,label

    def on_epoch_end(self):

        #Updates indexes after each epochs
        self.indices = np.arange(len(self.dataset))
        if self.shuffle == True:
            np.random.shuffle(self.indices)

'''
There is a wide range of possible file types which you can use to store data. HDF5 is one example.
HDF5 has some advantages over these data types. They other types are often slower and less compatible than HDF5 files.

An HDF5 based model is not too different compared to any other Keras model. 
In fact, the only differences are present at the start - namely, an extra import as well as 
a different way of loading the data. 

The following code uses h5py to load the training and testing data. From the HDF5 files, we retrieve the feature and label datasets.
'''
class DatasetMaker(object):

    def __init__(self,data_path):

        self.data_file = h5py.File(data_path)                                  #Gets the data path of the hash file

    def __len__(self):

        return len(self.data_file)

    def __getitem__(self,index):

        index += 1                                                               #Increment index
        video = self.data_file['video_'+str(index)]                              #Retrieve video according to the index
        feature = np.array(video['feature'][:])                                  #Retrieve the features
        label = np.array(video['label'][:])                                      #Retrieve the labels

        return feature,label,index

def get_loader(path, batch_size=5):

    dataset = DatasetMaker(path)
    train_dataset, test_dataset = train_test_split(dataset, test_size = 0.2)    #Splits dataset where 80% is trained and 20% is tested
    train_loader = DataGenerator(train_dataset)                                 #Feeds dataset to data generator

    return train_loader, test_dataset

In [3]:
import pprint
'''
This part of the code is for setting up basic configurations.

**kwargs allows you to pass keyworded variable length of arguments to a function. 
You should use **kwargs if you want to handle named arguments in a function.
'''
class Config():

    def __init__(self, **kwargs):

        self.data_path = 'fcsn_tvsum.h5'
        self.save_dir = 'save_dir'
        self.score_dir = 'score_dir'
        self.n_epochs = 5                       #Set number of epochs
        self.batch_size = 5                     #Set batch size

        for a,b in kwargs.items():
            setattr(self,a,b)

    def __repr__(self):

        config_str = 'Configurations\n' + pprint.pformat(self.__dict__)

        return config_str

In [4]:
'''
This part of the code includes the main encoder-decoder model.
'''
class BuildModel():

    def __init__(self, config = None, train_loader = None, test_dataset = None):

        #Initialization
        self.config = config
        self.train_loader = train_loader
        self.test_dataset = test_dataset

        if not os.path.exists(self.config.score_dir):
            os.mkdir(self.config.score_dir)             #Makes score directory, if it doesn't exist

        if not os.path.exists(self.config.save_dir):
            os.mkdir(self.config.save_dir)              #Makes save directory, if it doesn't exist

    def train(self):

        encoder_inputs = Input(shape = (320, 1024))     #Reshape the input array in-place by assigning a tuple of array dimensions to it

        '''
        The encoder utilizes bidirectional LSTM.
        Bidirectional wrapper creates two copies of given layer, forward_layer and backward_layer.
        The output of an LSTM cell or layer of cells is called the hidden state. We have 128 hidden states in LSTM. 
        Moreover, by setting the return_sequences attribute to True when defining the LSTM layer, It is possible 
        to access the hidden state output for each input time step. Each LSTM cell retains an internal state that 
        is not output, called the cell state, or c. Keras provides the return_state argument to the LSTM layer that 
        will provide access to the hidden state output (state_h) and the cell state (state_c).
        '''
        encoder_BidirectionalLSTM = Bidirectional(LSTM(128, return_sequences = True, return_state = True))
        encoder_out, fh, fc, bh, bc = encoder_BidirectionalLSTM(encoder_inputs)
        sh = Concatenate()([fh, bh])
        ch = Concatenate()([fc, bc])
        encoder_states = [sh, ch]

        '''
        The decoder utilizes LSTM.
        Here we have 256 hidden states.
        The encoder-decoder mechanism measures the importance of each frame.
        '''
        decoder_LSTM = LSTM(256, return_sequences = True)
        decoder_out = decoder_LSTM(encoder_out, initial_state = encoder_states)

        '''
        
        The core idea  of the attention layer is each time the model predicts an output word, 
        it only uses parts of the input where the most relevant information is concentrated 
        instead of the entire sequence. 
        '''
        attn_layer = Attention(name="Attention_Layer")
        attn_out =  attn_layer([encoder_out, decoder_out])

        #Concatenates the output of the attention layer and decoder
        decoder_concat_input = Concatenate(axis = -1, name = 'concat_layer')([decoder_out, attn_out])

        '''
        We use the TimeDistributed layer to process the output from the LSTM hidden layer.
        TimeDistributed layer applies the same layer to several inputs and produces one 
        output per input to get the result in time.The softmax function provides 
        a normalised result at each time step.
        '''
        dense = TimeDistributed(Dense(1, activation = 'softmax'))
        decoder_pred = dense(decoder_concat_input)

        model = Model(inputs = encoder_inputs, outputs = decoder_pred)

        '''
        An optimizer is one of the two arguments required for compiling a Keras model
        The Adaptive Moment Estimation is an algorithm for optimization technique for 
        gradient descent. The method is really efficient when working with large problem 
        involving a lot of data or parameters. It requires less memory and is efficient. 
        Intuitively, it is a combination of the gradient descent with momentum
        algorithm and the RMSP algorithm.

        learning_rate is a Tensor, floating point value, or a schedule that is a 
        tf.keras.optimizers.schedules.LearningRateSchedule, or a callable that takes no 
        arguments and returns the actual value to use.
        '''
        opt = tf.keras.optimizers.Adam(learning_rate=0.15)

        model.compile(loss = 'binary_crossentropy', optimizer = opt, metrics = ['accuracy'])    #Compliling the model
        model.summary()

        self.model = model

        #For evaluating reults for every epoch
        t = trange(self.config.n_epochs, desc = 'Epoch', ncols = 90)
        for epoch_i in t:

            model.fit(self.train_loader)

            ckpt_path = self.config.save_dir + '/epoch-{}.ckpt'.format(epoch_i)     #Saving the output of every epoch
            tqdm.write("Save parameters at {}".format(ckpt_path))
            model.save_weights(ckpt_path)                                           #Saving weights
            self.evaluate(epoch_i)

    def evaluate(self, epoch_i):

        out_dict = {}
        eval_arr = []
        table = PrettyTable()                                                       #Formatting the table
        table.title = 'Evaluation Result of epoch {}'.format(epoch_i)               #Saving evaluation results for every epoch
        table.field_names = ['ID', 'Precision', 'Recall', 'F-Score']
        table.float_format = '1.5'

        with h5py.File(self.config.data_path) as data_file:
            for feature, label, index in tqdm(self.test_dataset, desc = 'Evaluate', ncols = 90, leave = False):

                pred_score = self.model.predict(feature.reshape(-1,320,1024))
                video_info = data_file['video_'+str(index)]
                pred_score, pred_selected, pred_summary = eval.select_keyshots(video_info, pred_score)
                true_summary_arr = video_info['user_summary'][:]
                eval_res = [eval.eval_metrics(pred_summary, true_summary) for true_summary in true_summary_arr]
                eval_res = np.mean(eval_res, axis = 0).tolist()

                eval_arr.append(eval_res)
                table.add_row([index] + eval_res)

                out_dict[str(index)] = {
                'pred_score' : pred_score,
                'pred_selected' : pred_selected,
                'pred_summary' : pred_summary
                }

        score_save_path = self.config.score_dir + '/epoch-{}.json'.format(epoch_i)      #Saving scores in the score directory
        with open(score_save_path,'w') as f:
            tqdm.write('Save score at {}'.format(str(score_save_path)))
            json.dump(out_dict,f)
        eval_mean = np.mean(eval_arr, axis = 0).tolist()
        table.add_row(['mean'] + eval_mean)
        tqdm.write(str(table))

In [5]:
#Loading the dataset and training the model
train_config = Config()
train_loader, test_dataset = get_loader(train_config.data_path, batch_size = train_config.batch_size)
builder = BuildModel(train_config, train_loader, test_dataset)
builder.train()

/var/folders/fq/0p9vc1m94752qt15lmbkg_c40000gn/T/ipykernel_8696/2362700362.py:42: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  self.data_file = h5py.File(data_path)
2022-02-06 13:36:22.315489: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-06 13:36:22.343540: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fba7e531ea0 initialized for platform Host (this does not guarantee that XLA will be used). De

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 320, 1024)]  0                                            
__________________________________________________________________________________________________
bidirectional (Bidirectional)   [(None, 320, 256), ( 1180672     input_1[0][0]                    
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 256)          0           bidirectional[0][1]              
                                                                 bidirectional[0][3]              
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 256)          0           bidirectional[0][2]   

Epoch:   0%|                                                        | 0/5 [00:07<?, ?it/s]/var/folders/fq/0p9vc1m94752qt15lmbkg_c40000gn/T/ipykernel_8696/3161885467.py:64: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  with h5py.File(self.config.data_path) as data_file:

Evaluate:   0%|                                                    | 0/10 [00:00<?, ?it/s]

Save parameters at save_dir/epoch-0.ckpt



Epoch:  20%|█████████▌                                      | 1/5 [00:10<00:41, 10.41s/it]

...

Save score at score_dir/epoch-3.json
+--------------------------------------+
|     Evaluation Result of epoch 3     |
+------+-----------+---------+---------+
|  ID  | Precision |  Recall | F-Score |
+------+-----------+---------+---------+
|  25  |  0.46979  | 0.46678 | 0.46828 |
|  39  |  0.51981  | 0.52043 | 0.52010 |
|  6   |  0.60172  | 0.59447 | 0.59807 |
|  38  |  0.51379  | 0.51647 | 0.51504 |
|  34  |  0.57371  | 0.55339 | 0.56334 |
|  15  |  0.47622  | 0.47649 | 0.47633 |
|  8   |  0.45424  | 0.45046 | 0.45234 |
|  35  |  0.62108  | 0.61829 | 0.61966 |
|  12  |  0.62471  | 0.61559 | 0.62012 |
|  5   |  0.63819  | 0.64740 | 0.64241 |
| mean |  0.54933  | 0.54598 | 0.54757 |
+------+-----------+---------+---------+
8/8 [==============================] - 3s 435ms/step - loss: 12.4389 - accuracy: 0.1843


Evaluate:   0%|                                                    | 0/10 [00:00<?, ?it/s]

Save parameters at save_dir/epoch-4.ckpt



Epoch: 100%|████████████████████████████████████████████████| 5/5 [00:34<00:00,  6.84s/it]

Save score at score_dir/epoch-4.json
+--------------------------------------+
|     Evaluation Result of epoch 4     |
+------+-----------+---------+---------+
|  ID  | Precision |  Recall | F-Score |
+------+-----------+---------+---------+
|  25  |  0.46979  | 0.46678 | 0.46828 |
|  39  |  0.51981  | 0.52043 | 0.52010 |
|  6   |  0.60172  | 0.59447 | 0.59807 |
|  38  |  0.51379  | 0.51647 | 0.51504 |
|  34  |  0.57371  | 0.55339 | 0.56334 |
|  15  |  0.47622  | 0.47649 | 0.47633 |
|  8   |  0.45424  | 0.45046 | 0.45234 |
|  35  |  0.62108  | 0.61829 | 0.61966 |
|  12  |  0.62471  | 0.61559 | 0.62012 |
|  5   |  0.63819  | 0.64740 | 0.64241 |
| mean |  0.54933  | 0.54598 | 0.54757 |
+------+-----------+---------+---------+


In [6]:
'''
This part of the code allows us to use our custom input
on the created model and generate a video summary.
'''
#Initialization
h5_path = 'fcsn_tvsum.h5'
json_path = 'score_dir/epoch-4.json'
data_root = 'input.mp4'
save_dir = 'Results'

video_dir = os.path.join(data_root)
f_data = h5py.File(h5_path)
with open(json_path) as f:
    json_dict = json.load(f)
    ids = json_dict.keys()

#Using change points and scores for selecting frames
def get_keys(id):
    video_info = f_data['video_' + id]
    video_path = os.path.join(video_dir)
    cps = video_info['change_points'][()]
    pred_score = json_dict[id]['pred_score']
    pred_selected = json_dict[id]['pred_selected']

    video = cv2.VideoCapture(video_path)
    frames = []
    success, frame = video.read()
    while success:
        frames.append(frame)
        success, frame = video.read()
    frames = np.array(frames)
    keyshots = []
    for sel in pred_selected:
        for i in range(cps[sel][0], cps[sel][1]):
            keyshots.append(frames[i])
    keyshots = np.array(keyshots)

    write_path = os.path.join(save_dir,'summary.avi')                   #Saves the output as 'summary.avi' in the 'Results' folder
    video_writer = cv2.VideoWriter(write_path, cv2.VideoWriter_fourcc(*'XVID'), 24, keyshots.shape[2:0:-1])
    for frame in keyshots:
        video_writer.write(frame)
    video_writer.release()

#Function for generating the summary
def gen_summary():
    if not os.path.exists(save_dir):                                    #Making the 'Results' directory
        os.mkdir(save_dir)

    for id in ids:
        get_keys(id)


/var/folders/fq/0p9vc1m94752qt15lmbkg_c40000gn/T/ipykernel_8696/148203453.py:7: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  f_data = h5py.File(h5_path)


In [7]:
plt.switch_backend('agg')
gen_summary()

IndexError: index 6113 is out of bounds for axis 0 with size 5939